In [ ]:
from gensim.corpora import WikiCorpus
# wiki corpus


wiki_corpus = WikiCorpus('zhwiki-20221120-pages-articles-multistream.xml.bz2', dictionary={})
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

In [ ]:
import jieba
import time
from opencc import OpenCC  # 將字體轉換

startTime = time.time()

judge = open('./corpus_judge_ten_comma.txt', encoding='utf-8')
wiki = open('./wiki_text.txt', encoding='utf-8')
count_wiki = 1
count_judge = 1
with open('corpus_wiki_judge.txt', 'w', encoding='utf-8') as new_f:
    for content_wiki in wiki:
        new_f.write(''.join(content_wiki))
        print('wiki :', count_wiki)
        count_wiki+=1
    for content_judge in judge:
        new_f.write(''.join(content_judge))
        print('judge :', count_judge)
        count_judge+=1

print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")       
    

In [ ]:
import jieba
import time
from opencc import OpenCC  # 將字體轉換
# split word


# Initial
cc = OpenCC('s2t')  # 將簡體轉為繁體

# wiki_text_seg.txt 已斷詞
# wiki_text.txt 未斷詞

# Tokenize
startTime = time.time()
with open('./corpus_wiki_judge_comma_jieba.txt', 'w', encoding='utf-8') as new_f:
    with open('./corpus_wiki_judge.txt', 'r', encoding='utf-8') as f:
        # load 自訂義 dict
        dict = './dict_judge.txt'
        jieba.load_userdict(dict)
        
        for times, data in enumerate(f, 1):
            print('data num:', times)
            data = cc.convert(data)
            data = jieba.cut(data)
            data = [word for word in data if word != ' ']
            data = ' '.join(data)  # 將 list 轉成 string

            new_f.write(data)
            
print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")

In [1]:
from gensim.models import word2vec
import time
# train model

# Settings
seed = 666  # 亂數種子
sg = 0  # sg = 0 ( CBOW ) ; sg = 1 ( Skip-gram ) 
window_size = 10  # 週圍的詞要看多少範圍
vector_size = 250  # 轉成向量的維度
min_count = 1  # 詞頻少於 min_count 的詞不會參與訓練
workers = 12  # 訓練的並行數量，cpu
epochs = 8  # 訓練的迭代次數 iter
batch_words = 1000  # 每次給予多少詞量訓練

startTime = time.time()
train_data = word2vec.LineSentence('corpus_wiki_judge_comma_jieba.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

# save model
model.save('model_wiki_judge_d250_sg_s10_ep8.model')

print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")

總花費時間 : 46.20 分鐘


In [2]:
from gensim.models import word2vec
# test model

# load model
model = word2vec.Word2Vec.load('model_wiki_judge_d250_sg_s10_ep10')



# load model ( gensim )
# model = gensim.models.Word2Vec.load('word2vec.model')

print(model.wv['過失傷害'].shape)

for item in model.wv.most_similar('過失傷害'):
    print(item)

# 查看詞向量
model.wv['過失傷害']


# 查看最相近詞 positive / 最相近反義詞 negative  
# model.wv.most_similar(positive=['英國', '倫敦'], negative=['法國'], topn=20)


# 計算相似程度
model.wv.similarity('過失傷害', '車禍')




(250,)
('業務過失傷害', 0.8761041164398193)
('犯行', 0.7959913611412048)
('過失傷害致人重傷', 0.7841423749923706)
('業務過失傷害致人重傷', 0.7327824234962463)
('傷犯行', 0.7051565647125244)
('失重', 0.7017012238502502)
('過失致人重傷害', 0.7005434036254883)
('致重傷', 0.6871241927146912)
('有業務過', 0.6834397315979004)
('茶致', 0.6792193055152893)


0.57760245

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

wiki_text_seg = ('./wiki_text_seg.txt')
# Take word vectors
word_vectors = np.array([model[w] for w in wiki_text_seg])

# PCA, take the first 2 principal components
twodim = PCA().fit_transform(word_vectors)[:,:2]

# Draw
plt.figure(figsize=(6,6))
plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
for word, (x,y) in zip(words, twodim):
    plt.text(x+0.05, y+0.05, word)
plt.show()

### ============================

In [ ]:
import os.path

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


def display_pca_scatterplot(model, words):
    # Take word vectors
    word_vectors = np.array([model[w] for w in words])

    # PCA, take the first 2 principal components
    twodim = PCA().fit_transform(word_vectors)[:,:2]

    # Draw
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)
    plt.show()

def main():
    # Download pre-trained GloVe embeddings, turn into Word2Vec format
    glove_file = './data/glove.6B.100d.txt'
    word2vec_glove_file = './data/glove.6B.100d.word2vec.txt'
    if not os.path.isfile(word2vec_glove_file):
        glove2word2vec(glove_file, word2vec_glove_file)

    # Load model
    model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

    # Take word, return most similar words
    word = input('Similarity - Input word: ')
    while word != '':
        print(model.most_similar(word))
        word = input('Similarity - Input word: ')

    # Take a, b, c, return d for a : b = c : d
    a = input('Analogy a : b = c : d - Input a: ')
    b = input('Analogy a : b = c : d - Input b: ')
    c = input('Analogy a : b = c : d - Input c: ')
    while a != '' and b != '' and c != '':
        print(model.most_similar(positive=[c, b], negative=[a]))
        a = input('Analogy a : b = c : d - Input a: ')
        b = input('Analogy a : b = c : d - Input b: ')
        c = input('Analogy a : b = c : d - Input c: ')

    # Display scatter plot for words
    choice = input('2D word vector visualization - Input \'default\' for a default list of words, or a comma-separated list of words: ')
    while choice != '':
        if choice == 'default':
            words = ['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
                     'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
                     'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
                     'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
                     'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
                     'homework', 'assignment', 'problem', 'exam', 'test', 'class',
                     'school', 'college', 'university', 'institute']
            display_pca_scatterplot(model, words=words)
        else:
            display_pca_scatterplot(model, words=[word.strip() for word in choice.split(',')])
        choice = input('2D word vector visualization - Input \'default\' for a default list of words, or a comma-separated list of words: ')

if __name__ == '__main__':
    main()

In [ ]:
words = ['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
         'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
         'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
         'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
         'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
         'homework', 'assignment', 'problem', 'exam', 'test', 'class',
         'school', 'college', 'university', 'institute']
display_pca_scatterplot(model, words)